In [19]:
!pip install --quiet optuna

import optuna
import optuna.integration

     |████████████████████████████████| 308 kB 24.1 MB/s 
     |████████████████████████████████| 81 kB 7.8 MB/s 
     |████████████████████████████████| 209 kB 57.9 MB/s 
     |████████████████████████████████| 78 kB 6.3 MB/s 
     |████████████████████████████████| 112 kB 55.9 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
     |████████████████████████████████| 146 kB 43.8 MB/s 


In [71]:
import xgboost as xgb
import pandas as pd
import numpy as np
import sklearn
import sklearn.datasets
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## First  using the evaluation metric "Accuracy"

In [69]:
def objective(trial):
  X = pd.read_csv("/content/drive/My Drive/David_Enoma_PhD_Project/Xsubset.csv",header=None)
  y = pd.read_csv("/content/drive/My Drive/David_Enoma_PhD_Project/hapmap_phenotype_recoded.txt", sep = " ", header = None)
  y.replace([1,2], [0,1], inplace = True)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 234)
  dtrain = xgb.DMatrix(X_train, label = y_train)
  dtest = xgb.DMatrix(X_test, label = y_test)
  param = {
      "silent": 1,
      "objective": "binary:logistic",
      "eval_metric": "auc",
      # The booster parameter sets the type of learner. Usually this is either a tree 
      # or a linear function. In the case of trees, the model will consist of an ensemble 
      # of trees. For the linear booster, it will be a weighted sum of linear functions.
      "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
      "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
      "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
  }

  if param["booster"] == "gbtree" or param["booster"] == "dart":
    param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
    param["eta"]= trial.suggest_loguniform("eta", 1e-8, 1.0)
    param["gamma"]= trial.suggest_loguniform("gamma", 1e-8, 1.0)
    param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

  if param["booster"] == "dart":
    param["sample_type"]=trial.suggest_categorical("sample_type", ["uniform", "weighted"])
    param["normalize_type"]=trial.suggest_categorical("normalize_type", ["tree", "forest"])
    param["rate_drop"] = trial.suggest_loguniform("rate_drop", 1e-8, 1.0)
    param["skip_drop"] = trial.suggest_loguniform("skip_drop", 1e-8, 1.0)


    # Call Back for Pruning

  pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
  bst = xgb.train(param, dtrain, evals = [(dtest, "validation")], callbacks = [pruning_callback])
  preds = bst.predict(dtest)
  pred_labels = np.rint(preds)
  accuracy = accuracy_score(y_test, pred_labels)
  return accuracy

In [70]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials = 100, timeout=600)
trial = study.best_trial

print("Accuracy : %f", trial.value)
print("Best HyperParamteters : %f", trial.params)

[I 2022-06-02 17:12:50,838] A new study created in memory with name: no-name-2cf61db9-e1a6-4fe1-a8a4-41fb0e889d71


[0]	validation-auc:0.577922
[1]	validation-auc:0.637987
[2]	validation-auc:0.608766
[3]	validation-auc:0.519481
[4]	validation-auc:0.530844
[5]	validation-auc:0.571429
[6]	validation-auc:0.537338
[7]	validation-auc:0.576299
[8]	validation-auc:0.547078
[9]	validation-auc:0.535714


[I 2022-06-02 17:12:51,024] Trial 0 finished with value: 0.5833333333333334 and parameters: {'booster': 'gbtree', 'lambda': 0.0003447537140857252, 'alpha': 0.0051820331204337705, 'max_depth': 3, 'eta': 1.5086480712144296e-07, 'gamma': 1.1715942103882784e-08, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.5833333333333334.


[0]	validation-auc:0.599026
[1]	validation-auc:0.543831
[2]	validation-auc:0.527597
[3]	validation-auc:0.532468
[4]	validation-auc:0.550325
[5]	validation-auc:0.543831
[6]	validation-auc:0.537338
[7]	validation-auc:0.537338
[8]	validation-auc:0.540584
[9]	validation-auc:0.538961


[I 2022-06-02 17:12:51,201] Trial 1 finished with value: 0.5277777777777778 and parameters: {'booster': 'dart', 'lambda': 2.0833130078619587e-08, 'alpha': 5.521644699721967e-07, 'max_depth': 2, 'eta': 0.00037014413803287315, 'gamma': 0.0024492400102343027, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 1.864492287031283e-08, 'skip_drop': 3.4703271164922825e-07}. Best is trial 0 with value: 0.5833333333333334.


...

[0]	validation-auc:0.688312
[1]	validation-auc:0.678571
[2]	validation-auc:0.672078
[3]	validation-auc:0.672078
[4]	validation-auc:0.675325
[5]	validation-auc:0.678571
[6]	validation-auc:0.678571
[7]	validation-auc:0.672078
[8]	validation-auc:0.675325
[9]	validation-auc:0.685065


[I 2022-06-02 17:13:03,050] Trial 93 finished with value: 0.5833333333333334 and parameters: {'booster': 'gblinear', 'lambda': 0.9137835636225653, 'alpha': 3.929348201199006e-06}. Best is trial 2 with value: 0.6111111111111112.
[I 2022-06-02 17:13:03,144] Trial 94 pruned. Trial was pruned at iteration 0.


[0]	validation-auc:0.668831
[1]	validation-auc:0.675325
[2]	validation-auc:0.681818
[3]	validation-auc:0.665584
[4]	validation-auc:0.652597
[5]	validation-auc:0.655844
[6]	validation-auc:0.659091
[7]	validation-auc:0.662338
[8]	validation-auc:0.662338
[9]	validation-auc:0.662338


[I 2022-06-02 17:13:03,320] Trial 95 finished with value: 0.5833333333333334 and parameters: {'booster': 'gblinear', 'lambda': 0.5078823815432375, 'alpha': 1.4413684163468104e-08}. Best is trial 2 with value: 0.6111111111111112.
[I 2022-06-02 17:13:03,415] Trial 96 pruned. Trial was pruned at iteration 0.
[I 2022-06-02 17:13:03,521] Trial 97 pruned. Trial was pruned at iteration 0.
[I 2022-06-02 17:13:03,622] Trial 98 pruned. Trial was pruned at iteration 0.
[I 2022-06-02 17:13:03,730] Trial 99 pruned. Trial was pruned at iteration 0.


Accuracy : %f 0.6111111111111112
Best HyperParamteters : %f {'booster': 'gblinear', 'lambda': 0.030475097247617086, 'alpha': 3.753658128510404e-05}


### Accuracy 
0.6111111111111112

### Best HyperParamteters 
'booster': 'gblinear', 

'lambda': 0.030475097247617086, 

'alpha': 3.753658128510404e-05

## For a problem as sensitive as Cancer, Evaluation metrics such as "Recall" or "F1 Score" are more important than "Accuracy" because the focus is on ensuring the highest number of people who have cancer are predicted to have cancer and so I will be using "Recall" as my evaluation metric here

In [74]:
def objective(trial):
  X = pd.read_csv("/content/drive/My Drive/David_Enoma_PhD_Project/Xsubset.csv",header=None)
  y = pd.read_csv("/content/drive/My Drive/David_Enoma_PhD_Project/hapmap_phenotype_recoded.txt", sep = " ", header = None)
  y.replace([1,2], [0,1], inplace = True)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 234)
  dtrain = xgb.DMatrix(X_train, label = y_train)
  dtest = xgb.DMatrix(X_test, label = y_test)
  param = {
      "verbosity": 0,
      # logistic regression for binary classification, output probability
      "objective": "binary:logistic",
      # use exact for small dataset.
      "tree_method": "auto",
      "eval_metric": "auc",
      # The booster parameter sets the type of learner. Usually this is either a tree 
      # or a linear function. In the case of trees, the model will consist of an ensemble 
      # of trees. For the linear booster, it will be a weighted sum of linear functions.
      # defines booster, gblinear for linear functions.
      "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
      # L2 regularization weight.
      "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log = True), 
      # L1 regularization weight.
      "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log = True),
      # sampling ratio for training data.
      "subsample": trial.suggest_float("subsample", 0.2, 1.0),
      # sampling according to each tree.
      "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
  }

  if param["booster"] == "gbtree" or param["booster"] == "dart":
    # maximum depth of the tree, signifies complexity of the tree.
    param["max_depth"] = trial.suggest_int("max_depth", 1, 7, step=2)
    # minimum child weight, larger the term more conservative the tree.
    param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
    # "eta" shrinks the feature weights to make the boosting process more conservative.
    param["eta"]= trial.suggest_float("eta", 1e-8, 1.0, log = True)
    # Minimum loss reduction required to make a further partition on a leaf node of the tree. 
    # The larger gamma is, the more conservative the algorithm will be.
    param["gamma"]= trial.suggest_float("gamma", 1e-8, 1.0, log = True)
    #  Controls the way new nodes are added to the tree.
    param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    
  if param["booster"] == "dart":
    # Type of sampling algorithm.
    param["sample_type"]=trial.suggest_categorical("sample_type", ["uniform", "weighted"])
    # Type of normalization algorithm.
    param["normalize_type"]=trial.suggest_categorical("normalize_type", ["tree", "forest"])
    # Dropout rate (a fraction of previous trees to drop during the dropout).
    param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log = True)
    # Probability of skipping the dropout procedure during a boosting iteration.
    param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log = True)


  # Call Back for Pruning
  # Pruning is used to terminate unpromising trials early, so that computing time can be used for trials that show more potential.
  pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
  bst = xgb.train(param, dtrain, evals = [(dtest, "validation")], callbacks = [pruning_callback])
  preds = bst.predict(dtest)
  # np.rint means return integer. It rounds up elements to the nearest integer. 
  # This is done so we can use accuracy_score because "binary:logistic" outputs probability not discrete numbers [0,1] but the probability of each sample being 1
  pred_labels = np.rint(preds)
  Recall = recall_score(y_test, pred_labels)
  return Recall

In [73]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials = 100, timeout=600)
trial = study.best_trial

print("Recall : %f", trial.value)
print("Best HyperParamteters : %f", trial.params)

[I 2022-06-02 17:38:47,803] A new study created in memory with name: no-name-51a4781b-0934-418f-a61f-a177cffb35d4


[0]	validation-auc:0.626623
[1]	validation-auc:0.63961
[2]	validation-auc:0.62987
[3]	validation-auc:0.603896
[4]	validation-auc:0.594156
[5]	validation-auc:0.600649
[6]	validation-auc:0.587662
[7]	validation-auc:0.581169
[8]	validation-auc:0.584416
[9]	validation-auc:0.577922


[I 2022-06-02 17:38:48,156] Trial 0 finished with value: 0.7857142857142857 and parameters: {'booster': 'gblinear', 'lambda': 5.491115673572802e-08, 'alpha': 0.002810245821911446, 'subsample': 0.587167981915971, 'colsample_bytree': 0.37519211627122084}. Best is trial 0 with value: 0.7857142857142857.


[0]	validation-auc:0.548701
[1]	validation-auc:0.571429
[2]	validation-auc:0.623377
[3]	validation-auc:0.626623
[4]	validation-auc:0.631494
[5]	validation-auc:0.623377
[6]	validation-auc:0.616883
[7]	validation-auc:0.61526
[8]	validation-auc:0.618506
[9]	validation-auc:0.568182


[I 2022-06-02 17:38:48,503] Trial 1 finished with value: 0.7142857142857143 and parameters: {'booster': 'dart', 'lambda': 0.00027626960218040563, 'alpha': 0.5867635045050236, 'subsample': 0.6608825479286826, 'colsample_bytree': 0.38627499741087096, 'max_depth': 1, 'min_child_weight': 5, 'eta': 1.4474382771489371e-06, 'gamma': 4.5679746479102705e-07, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 0.008106881402335753, 'skip_drop': 0.013935454738290267}. Best is trial 0 with value: 0.7857142857142857.


...

[0]	validation-auc:0.685065
[1]	validation-auc:0.681818
[2]	validation-auc:0.685065
[3]	validation-auc:0.678571
[4]	validation-auc:0.678571
[5]	validation-auc:0.681818
[6]	validation-auc:0.675325
[7]	validation-auc:0.672078
[8]	validation-auc:0.672078
[9]	validation-auc:0.675325


[I 2022-06-02 17:39:00,421] Trial 88 finished with value: 0.8571428571428571 and parameters: {'booster': 'gblinear', 'lambda': 0.8373727678018571, 'alpha': 8.607805681861056e-05, 'subsample': 0.3129248379752191, 'colsample_bytree': 0.5784009655497335}. Best is trial 2 with value: 1.0.
[I 2022-06-02 17:39:00,540] Trial 89 pruned. Trial was pruned at iteration 0.
[I 2022-06-02 17:39:00,653] Trial 90 pruned. Trial was pruned at iteration 0.
[I 2022-06-02 17:39:00,758] Trial 91 pruned. Trial was pruned at iteration 0.
[I 2022-06-02 17:39:00,856] Trial 92 pruned. Trial was pruned at iteration 0.


[0]	validation-auc:0.685065
[1]	validation-auc:0.681818
[2]	validation-auc:0.675325
[3]	validation-auc:0.668831
[4]	validation-auc:0.668831
[5]	validation-auc:0.662338
[6]	validation-auc:0.665584
[7]	validation-auc:0.672078
[8]	validation-auc:0.665584
[9]	validation-auc:0.672078


[I 2022-06-02 17:39:01,042] Trial 93 finished with value: 0.8571428571428571 and parameters: {'booster': 'gblinear', 'lambda': 0.6891434997345821, 'alpha': 6.36119943223149e-07, 'subsample': 0.6174021642034098, 'colsample_bytree': 0.3846307632792385}. Best is trial 2 with value: 1.0.
[I 2022-06-02 17:39:01,144] Trial 94 pruned. Trial was pruned at iteration 0.
[I 2022-06-02 17:39:01,244] Trial 95 pruned. Trial was pruned at iteration 0.
[I 2022-06-02 17:39:01,340] Trial 96 pruned. Trial was pruned at iteration 0.
[I 2022-06-02 17:39:01,438] Trial 97 pruned. Trial was pruned at iteration 0.
[I 2022-06-02 17:39:01,556] Trial 98 pruned. Trial was pruned at iteration 0.
[I 2022-06-02 17:39:01,664] Trial 99 pruned. Trial was pruned at iteration 0.


Recall : %f 1.0
Best HyperParamteters : %f {'booster': 'dart', 'lambda': 8.669790267915281e-07, 'alpha': 1.1536382592082632e-08, 'subsample': 0.28469669085250404, 'colsample_bytree': 0.5566590888822147, 'max_depth': 7, 'min_child_weight': 10, 'eta': 0.003933096882701822, 'gamma': 0.0018015367008443472, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 8.829825957910605e-06, 'skip_drop': 5.189332052983798e-07}


### Recall
1.0

### Best HyperParamteters
'booster': 'dart', 

'lambda': 8.669790267915281e-07, 

'alpha': 1.1536382592082632e-08, 

'subsample': 0.28469669085250404, 

'colsample_bytree': 0.5566590888822147, 

'max_depth': 7, 

'min_child_weight': 10, 

'eta': 0.003933096882701822, 

'gamma': 0.0018015367008443472, 

'grow_policy': 'lossguide', 

'sample_type': 'weighted', 

'normalize_type': 'forest', 

'rate_drop': 8.829825957910605e-06, 

'skip_drop': 5.189332052983798e-07}